In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers torch

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from transformers import pipeline
import os

In [5]:
path_articles = "/content/drive/Shareddrives/information project/Information Disorder Project/Fase 1 - Raccolta articoli OSINT/Prove Franci/google_news_svb_netflix_FINAL_CLEAN.csv"

articles_df = pd.read_csv(path_articles)

display(articles_df.head())


,case,title,source,published_utc,link,query,date
0,SVB,Silicon Valley Bank and Signature Bank taken o...,Norton Rose Fulbright,2023-03-10 08:00:00+00:00,https://news.google.com/rss/articles/CBMizwFBV...,"""Silicon Valley Bank"" regulators",2023-03-10
1,SVB,"Silicon Valley Bank Closed by Regulators, FDIC...",The Wall Street Journal,2023-03-10 08:00:00+00:00,https://news.google.com/rss/articles/CBMizgNBV...,"""Silicon Valley Bank"" regulators",2023-03-10
2,SVB,Regulators take over Silicon Valley Bank - Ban...,Banking Dive,2023-03-10 08:00:00+00:00,https://news.google.com/rss/articles/CBMitwFBV...,"""Silicon Valley Bank"" regulators",2023-03-10
3,SVB,Silicon Valley Bank has officially failed afte...,Fortune,2023-03-10 08:00:00+00:00,https://news.google.com/rss/articles/CBMiigFBV...,"""Silicon Valley Bank"" regulators",2023-03-10
4,SVB,Silicon Valley Bank shut down by regulators - ...,Fox Business,2023-03-10 08:00:00+00:00,https://news.google.com/rss/articles/CBMiiAFBV...,"""Silicon Valley Bank"" regulators",2023-03-10


In [6]:
distilbert_multi_sentiment = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [7]:
def distilbert_multi_score(text):
    try:
        result = distilbert_multi_sentiment(str(text)[:512])[0]
        label = result["label"]      # es. "1 star", "5 stars"
        stars = int(label[0])

        # mapping stelle → [-1, +1]
        if stars <= 2:
            return - (3 - stars) / 2
        elif stars == 3:
            return 0.0
        else:
            return (stars - 3) / 2
    except:
        return 0.0


In [8]:
articles_df["distilbert_multi_score"] = articles_df["title"].apply(distilbert_multi_score)

In [9]:
articles_df[["case", "title", "distilbert_multi_score"]].head(10)


,case,title,distilbert_multi_score
0,SVB,Silicon Valley Bank and Signature Bank taken o...,-1.0
1,SVB,"Silicon Valley Bank Closed by Regulators, FDIC...",-1.0
2,SVB,Regulators take over Silicon Valley Bank - Ban...,-1.0
3,SVB,Silicon Valley Bank has officially failed afte...,-1.0
4,SVB,Silicon Valley Bank shut down by regulators - ...,-1.0
5,SVB,"Regulators shut down Silicon Valley Bank, capp...",-1.0
6,SVB,US regulators shut down Silicon Valley Bank - ...,-1.0
7,SVB,Silicon Valley Bank is seized by U.S. after hi...,-1.0
8,SVB,Silicon Valley Bank Collapse Leaves Tech Indus...,-1.0
9,SVB,SVB is largest bank failure since 2008 financi...,-1.0


In [13]:
local_path = "/content/google_news_distilbert_multi.csv"
articles_df.to_csv(local_path, index=False)

print("File salvato in Colab:")
print(local_path)


File salvato in Colab:
/content/google_news_distilbert_multi.csv
